In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.append('..')
from dataset import batcher

In [53]:
MODEL_DIR = '../saved_models/grid_search/'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']
# COUNTRIES = ['ethiopia-2018', 'nigeria-2018', 'malawi-2016']
COUNTRIES = ['ethiopia-2018', 'nigeria-2018', 'pollution-2018']

In [40]:
# Get test split
bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP', 'VIIRS', 'DELTA_TEMP', 'CO']
# bands = ['VIIRS']
def batch(country_year):
    test_batcher = batcher.Batcher(bands, country_year=country_year, bucket=False, shuffle=False, split='custom', repeat=1, batch_size=3000).get_dataset()
    x_true = []
    y_true = []
    for x, y in test_batcher:
        x_true.append(x)
        y_true.append(y)

    y_true = np.squeeze(y_true)
    return x_true, y_true

In [41]:
# Load model
def load_model_from_dir(model_name):
    model_path = MODEL_DIR + model_name + '.h5'
    model = tf.keras.models.load_model(model_path, compile=False)
    
    return model

In [42]:
# Testing purposes only
# grid_search_results = pd.DataFrame(columns=['experiment', 'epoch', 'learning_rate', 'r2'])
# results = {'experiment': "imagery_sample_cnn_regression_0.01_40", 'epoch': 50, 'learning_rate': 1e-2, 'r2': 0.23}
# grid_search_results = grid_search_results.append(results, ignore_index = True)
# display(grid_search_results)

In [56]:
# Hyperparamters tested
lrs = [1e-1, 1e-2, 1e-3, 1e-4]
models = ['sample_cnn', 'deep_sample_cnn', 'sample_vgg']

"""
Hyperparamters to test:

"""

def run_grid_search(country_year):
    grid_search_results = pd.DataFrame(columns=['experiment', 'model', 'learning_rate', 'r2'])
    x_true, y_true = batch(country_year)

    for m in models:
        for l in lrs:
            experiment_name='imagery_' + m + '_regression' + '_' + str(l)

            # Set hyperparamters
            opt = tf.keras.optimizers.Adam(learning_rate=l)

            # Get consumption predictions
            trained_model = load_model_from_dir(experiment_name)
            trained_model.compile(
                        loss=LOSS,
                        optimizer=opt,
                        metrics=METRICS,
            )

            y_preds = trained_model.predict_on_batch(x_true)
            y_preds = np.squeeze(y_preds)
            r2 = pearsonr(y_true, y_preds)[0]

            results = {'experiment': experiment_name, 'model': m, 'learning_rate': l, 'r2': r2}
            grid_search_results = grid_search_results.append(results, ignore_index = True)
            
    return grid_search_results
# epochs = [10, 50, 100, 200]

In [57]:
for country_year in COUNTRIES:  
    country_grid_search_results = run_grid_search(country_year)
    print(f'{country_year}:')
    print(country_grid_search_results.sort_values(by=['r2']))

2023-02-23 17:24:45.996826: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2781914079.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 17:24:50.938626: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2781914079.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 17:24:54.238049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.

ethiopia-2018:
                                   experiment            model  learning_rate  \
7   imagery_deep_sample_cnn_regression_0.0001  deep_sample_cnn         0.0001   
9          imagery_sample_vgg_regression_0.01       sample_vgg         0.0100   
4      imagery_deep_sample_cnn_regression_0.1  deep_sample_cnn         0.1000   
0           imagery_sample_cnn_regression_0.1       sample_cnn         0.1000   
2         imagery_sample_cnn_regression_0.001       sample_cnn         0.0010   
1          imagery_sample_cnn_regression_0.01       sample_cnn         0.0100   
8           imagery_sample_vgg_regression_0.1       sample_vgg         0.1000   
3        imagery_sample_cnn_regression_0.0001       sample_cnn         0.0001   
10        imagery_sample_vgg_regression_0.001       sample_vgg         0.0010   
5     imagery_deep_sample_cnn_regression_0.01  deep_sample_cnn         0.0100   
6    imagery_deep_sample_cnn_regression_0.001  deep_sample_cnn         0.0010   
11       imag

2023-02-23 17:25:41.048890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2781914079.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 17:25:44.461681: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2781914079.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 17:25:47.774998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.

nigeria-2018:
                                   experiment            model  learning_rate  \
7   imagery_deep_sample_cnn_regression_0.0001  deep_sample_cnn         0.0001   
11       imagery_sample_vgg_regression_0.0001       sample_vgg         0.0001   
3        imagery_sample_cnn_regression_0.0001       sample_cnn         0.0001   
4      imagery_deep_sample_cnn_regression_0.1  deep_sample_cnn         0.1000   
9          imagery_sample_vgg_regression_0.01       sample_vgg         0.0100   
5     imagery_deep_sample_cnn_regression_0.01  deep_sample_cnn         0.0100   
8           imagery_sample_vgg_regression_0.1       sample_vgg         0.1000   
1          imagery_sample_cnn_regression_0.01       sample_cnn         0.0100   
6    imagery_deep_sample_cnn_regression_0.001  deep_sample_cnn         0.0010   
2         imagery_sample_cnn_regression_0.001       sample_cnn         0.0010   
10        imagery_sample_vgg_regression_0.001       sample_vgg         0.0010   
0           im

2023-02-23 17:26:30.711196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2781914079.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 17:26:34.297547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2781914079.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 17:26:37.601018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.

pollution-2018:
                                   experiment            model  learning_rate  \
10        imagery_sample_vgg_regression_0.001       sample_vgg         0.0010   
0           imagery_sample_cnn_regression_0.1       sample_cnn         0.1000   
7   imagery_deep_sample_cnn_regression_0.0001  deep_sample_cnn         0.0001   
4      imagery_deep_sample_cnn_regression_0.1  deep_sample_cnn         0.1000   
9          imagery_sample_vgg_regression_0.01       sample_vgg         0.0100   
2         imagery_sample_cnn_regression_0.001       sample_cnn         0.0010   
6    imagery_deep_sample_cnn_regression_0.001  deep_sample_cnn         0.0010   
8           imagery_sample_vgg_regression_0.1       sample_vgg         0.1000   
1          imagery_sample_cnn_regression_0.01       sample_cnn         0.0100   
11       imagery_sample_vgg_regression_0.0001       sample_vgg         0.0001   
3        imagery_sample_cnn_regression_0.0001       sample_cnn         0.0001   
5     imager

/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2781914079.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)


In [ ]:
"""
Results:

"""